#Setup for Model1


In [ ]:
!pip install tensorflow
!pip install keras
!pip install tensorflow-hub
!pip install tensorflow-text
!pip install --upgrade tf_keras

#MODEL 1


In [ ]:
import csv
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def convert_json_to_csv(json_files, csv_file):
    data = []
    for json_file in json_files:
        with open(json_file, 'r') as f:
            data.extend(json.load(f))

    with open(csv_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['id', 'labels', 'text'])
        writer.writeheader()
        for item in data:
            writer.writerow(item)

convert_json_to_csv(['Training Set Task1.json', 'Test Set Task1.json'], 'output.csv')
df = pd.read_csv('output.csv')

In [ ]:
import pandas as pd

columns = [
    'text',
    'Appeal to authority',
    'Appeal to fear/prejudice',
    'Black-and-white Fallacy/Dictatorship',
    'Causal Oversimplification',
    'Doubt',
    'Exaggeration/Minimisation',
    'Flag-waving',
    'Glittering generalities (Virtue)',
    'Loaded Language',
    'Misrepresentation of Someone\'s Position (Straw Man)',
    'Name calling/Labeling',
    'Obfuscation, Intentional vagueness, Confusion',
    'Presenting Irrelevant Data (Red Herring)',
    'Reductio ad hitlerum',
    'Repetition',
    'Slogans',
    'Smears',
    'Thought-terminating cliché',
    'Whataboutism',
    'Bandwagon'
]

new_df = pd.DataFrame(columns=columns)
for index, row in df.iterrows():
    row_values = {'text': row['text']}
    for label in columns[1:]:
        if label in row['labels']:
            row_values[label] = 1
        else:
            row_values[label] = 0

    new_df.loc[len(new_df)] = row_values

final = pd.concat([df['text'], new_df.drop(columns=['text'])], axis=1)


In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

bert_preprocess = hub.KerasLayer('https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3')
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-12-h-768-a-12/versions/2")


In [ ]:

def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

embedded_vectors = []
df_values = df.values
for i in range(len(df_values)):
  embedded_vector = get_sentence_embedding([df_values[i][2]])
  embedded_vectors.append(embedded_vector)



In [ ]:
final['EMBEDDED'] = embedded_vectors


def tensor_to_array(tensor):
    return tensor.numpy()

final['EMBEDDED'] = final['EMBEDDED'].apply(tensor_to_array)

In [ ]:
import pandas as pd
import numpy as np

embedded_df = pd.DataFrame()

for index, row in final.iterrows():
    embedded_row = pd.DataFrame([np.squeeze(row['EMBEDDED'])], columns=[f'embedded_{i}' for i in range(len(np.squeeze(row['EMBEDDED'])))])
    embedded_df = pd.concat([embedded_df, embedded_row], ignore_index=True)



In [ ]:
final = final.drop('text', axis =1)


In [ ]:
import IPython.display

for i, column in enumerate(final.columns):
    new_df = pd.concat([final[column], embedded_df], axis=1)

combined_df = pd.concat([embedded_df, final], axis=1)
combined_df = combined_df.drop('EMBEDDED', axis=1)
final = final.drop('EMBEDDED', axis=1)


combined_df

KeyError: "['EMBEDDED'] not found in axis"

In [ ]:
from sklearn.model_selection import train_test_split

y_dropper = final.columns
final = final.drop('')
y_keep =
train_ml, test_ml = train_test_split(combined_df, test_size=0.3, random_state=42)  # Stratify based on the target column

x_train_ml = train_ml.drop(y_dropper, axis=1)
y_train_ml = train_ml[y_dropper]

x_test_ml = test_ml.drop(y_dropper, axis=1)
y_test_ml = test_ml[y_dropper]




In [ ]:
y_train_ml

,Appeal to authority,Appeal to fear/prejudice,Black-and-white Fallacy/Dictatorship,Causal Oversimplification,Doubt,Exaggeration/Minimisation,Flag-waving,Glittering generalities (Virtue),Loaded Language,Misrepresentation of Someone's Position (Straw Man),Name calling/Labeling,"Obfuscation, Intentional vagueness, Confusion",Presenting Irrelevant Data (Red Herring),Reductio ad hitlerum,Repetition,Slogans,Smears,Thought-terminating cliché,Whataboutism,Bandwagon
485,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
292,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
172,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
449,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
360,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
270,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
435,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0


In [ ]:
x_train_ml

,embedded_0,embedded_1,embedded_2,embedded_3,embedded_4,embedded_5,embedded_6,embedded_7,embedded_8,embedded_9,...,embedded_759,embedded_760,embedded_761,embedded_762,embedded_763,embedded_764,embedded_765,embedded_766,embedded_767,text
485,-0.085058,-0.078582,0.869622,-0.257848,0.351566,-0.995727,0.439743,0.984074,-0.474738,0.194668,...,0.298589,0.795813,-0.769860,-0.989406,0.560208,-0.205174,-0.998800,0.372030,0.995645,"So, we can all gather to do this..\n\nBUT NOT ..."
292,-0.539396,-0.477195,0.006730,-0.330868,0.554595,-1.000000,0.817428,0.953124,-0.780424,0.603461,...,0.628124,0.995751,-0.159940,-0.955899,0.208755,-0.571810,-0.997633,0.691139,0.999686,This is not Baghdad.\nThis is not Benghazi.\nT...
172,-0.147695,-0.148959,0.946392,-0.150558,0.043216,0.850205,0.252953,0.904829,-0.048200,0.213991,...,0.394151,0.275247,0.572892,-0.999200,0.930217,-0.051216,-0.982076,0.197851,0.999127,They say you can't fix STUPID\nTURNS OUT You C...
449,-0.048873,-0.514379,0.994174,-0.234785,-0.210245,-0.852455,0.049686,0.891942,-0.464947,-0.232421,...,-0.262342,0.666370,0.332441,0.010829,0.995172,-0.253044,0.572774,0.645078,0.999748,A July 4th message from Joe:\n\nMerry Christmas\n
360,-0.046054,-0.493315,0.571409,-0.720426,0.698835,-1.000000,0.336288,-0.502442,-0.539559,0.703644,...,0.558616,0.998197,0.722339,-0.021784,0.912550,-0.568402,0.792160,0.770903,0.995256,Losers can keep watching sweaty NFL thugs supp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.365229,-0.017259,0.909588,0.013976,-0.030304,-0.999999,0.044545,0.993279,-0.464484,-0.160901,...,0.124742,0.907714,-0.296420,-0.991930,0.808448,-0.439646,-0.963777,0.133364,0.991057,FOR THE LOVE OF GOD ARREST SOMEBODY! \n
270,-0.022727,-0.167816,0.751035,-0.055271,0.201100,-0.999680,0.314545,0.966420,-0.085403,-0.407845,...,0.304765,0.776733,0.972160,-0.997722,0.733323,0.059952,-0.993318,0.125693,0.987019,NANCY THINKS THIS IS MORBIDLY OBESE. \nWHAT DO...
860,-0.095103,-0.074880,-0.566873,-0.370488,0.239618,-0.999683,0.212006,-0.093161,-0.412995,0.328920,...,0.383893,0.807325,-0.081709,-0.852583,-0.210286,-0.139637,-0.943368,0.504937,0.975536,"MISSING\n\nHAVE YOU SEEN ME?\nIf found, contac..."
435,0.010069,-0.363047,0.832051,-0.140532,0.312486,-0.997328,-0.001747,0.987758,-0.426070,0.125964,...,0.517132,0.472051,0.994578,-0.634412,0.898653,0.041514,-0.946851,0.451255,0.975550,Her choices of great role models - perverted m...


In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report
import numpy as np
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_ml.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(20, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

total_samples = len(y_train_ml)
class_counts = np.sum(y_train_ml, axis=0)
class_weights = {i: total_samples / (2 * class_counts[i]) for i in range(20)}

model.fit(x_train_ml, y_train_ml,
          batch_size=32,
          epochs=100,
          validation_data=(x_test_ml, y_test_ml),
          class_weight=class_weights,
          verbose=0)

y_pred_ml = model.predict(x_test_ml)
y_pred_ml = (y_pred_ml > 0.2).astype(int)

for i in range(20):
    print(f"Label {i+1} classification report:")
    print(classification_report(y_test_ml.iloc[:, i], y_pred_ml[:, i]))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
print(y_train_ml.shape)

(621, 20)


In [ ]:
for i in range(y_test_ml.shape[1]):
    print(f"y_test_ml[:, {i}]:", y_test_ml.iloc[:, i])
    print(f"y_pred_ml[:, {i}]:", y_pred_ml[:, i])

y_test_ml[:, 0]: 280    0
434    0
39     0
417    0
584    0
      ..
453    0
631    0
692    0
740    0
429    0
Name: Appeal to authority, Length: 267, dtype: int64
y_pred_ml[:, 0]: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
y_test_ml[:, 1]: 280    0
434    0
39     0
417    1
584    0
      ..
453    0
631    0
692    0
740    0
429    0
Name: Appeal to fear/prejudice, Length: 267, dtype: int64
y_pred_ml[:, 1]: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 

#Model 2


In [ ]:
!pip install keras
!pip install tensorflow
!pip install pandas
!pip install Pillow

In [ ]:
from google.colab import drive
import os
import pandas as pd


# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import csv
import io
from sklearn.preprocessing import LabelEncoder
from google.colab import files


def convert_json_to_csv(json_files, csv_file):
    data = []
    for json_file in json_files:
        with open(json_file, 'r') as f:
            json_data = json.load(f)
            for entry in json_data:
              if 'labels' in entry and isinstance(entry['labels'], list):
                entry['labels'] = ', '.join(entry['labels'])
              data.append(entry)

    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        if data:
            fieldnames = data[0].keys()  # Dynamically determine fieldnames from data
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for item in data:
                writer.writerow(item)

json_train_files = ['training_set_task3.json']
json_test_files = ['test_set_task3.json']
output_train_csv_file = '/content/output_train.csv'
output_test_csv_file = '/content/output_test.csv'
convert_json_to_csv(json_train_files, output_train_csv_file)
convert_json_to_csv(json_test_files, output_test_csv_file)
train_df = pd.read_csv(output_train_csv_file)
test_df = pd.read_csv(output_test_csv_file)
print(train_df.head())
print(test_df.head())

    id                                             labels  \
0  128  Black-and-white Fallacy/Dictatorship, Name cal...   
1  189             Reductio ad hitlerum, Smears, Transfer   
2   96  Appeal to fear/prejudice, Loaded Language, Nam...   
3  154  Causal Oversimplification, Glittering generali...   
4   15  Flag-waving, Misrepresentation of Someone's Po...   

                                                text          image  
0    THERE ARE ONLY TWO GENDERS\n\nFEMALE \n\nMALE\n  128_image.png  
1                           This is not an accident!  189_image.png  
2  SO BERNIE BROS HAVEN'T COMMITTED VIOLENCE EH?\...   96_image.png  
3  PATHETIC\n\nThe Cowardly Asshole\nWeak Failure...  154_image.png  
4  WHO TRUMP REPRESENTS\n\nWHO DEMOCRATS REPRESENT\n   15_image.png  
            id                                             labels  \
0  705_batch_2   Name calling/Labeling, Slogans, Smears, Transfer   
1  706_batch_2  Appeal to (Strong) Emotions, Appeal to fear/pr...   
2  710

In [ ]:
train_df['labels'].value_counts()

labels
Smears                                                                                                                         82
Loaded Language, Name calling/Labeling, Smears                                                                                 67
Loaded Language, Smears                                                                                                        51
Name calling/Labeling, Smears                                                                                                  32
Loaded Language                                                                                                                20
                                                                                                                               ..
Smears, Thought-terminating cliché                                                                                              1
Loaded Language, Misrepresentation of Someone's Position (Straw Man), Name calling/

In [ ]:
test_df['labels'].value_counts()

labels
Doubt, Loaded Language                                                                  9
Doubt                                                                                   9
Smears                                                                                  9
Exaggeration/Minimisation, Smears                                                       8
Loaded Language, Name calling/Labeling, Smears                                          6
                                                                                       ..
Appeal to (Strong) Emotions, Repetition, Slogans, Transfer                              1
Glittering generalities (Virtue), Loaded Language, Name calling/Labeling                1
Exaggeration/Minimisation, Glittering generalities (Virtue), Transfer                   1
Doubt, Glittering generalities (Virtue), Loaded Language, Thought-terminating cliché    1
Name calling/Labeling, Smears                                                           1
Nam

In [ ]:
train_dictionary = {}
for l in train_df['labels']:
  l = str(l)
  string = l.split(', ')
  # Obfuscation, Intentional vagueness, Confusion
  for s in string:
    if (s == "Intentional vagueness" or s == "Obfuscation" or s == "Confusion" ):
      train_dictionary[s] = train_dictionary.get("Obfuscation, Intentional vagueness, Confusion", 0) + 1
    else:
      train_dictionary[s] = train_dictionary.get(s, 0) + 1

In [ ]:
test_dictionary = {}
for l in test_df['labels']:
  l = str(l)
  string = l.split(', ')
  for s in string:
    test_dictionary[s] = test_dictionary.get(s, 0) + 1

In [ ]:
y_train_actual = list(train_dictionary.keys())
print(y_train_actual)

['Black-and-white Fallacy/Dictatorship', 'Name calling/Labeling', 'Smears', 'Reductio ad hitlerum', 'Transfer', 'Appeal to fear/prejudice', 'Loaded Language', 'Slogans', 'Causal Oversimplification', 'Glittering generalities (Virtue)', 'Flag-waving', "Misrepresentation of Someone's Position (Straw Man)", 'Exaggeration/Minimisation', 'Repetition', 'Appeal to (Strong) Emotions', 'Doubt', 'Obfuscation', 'Intentional vagueness', 'Confusion', 'Whataboutism', 'Thought-terminating cliché', 'Presenting Irrelevant Data (Red Herring)', 'Appeal to authority', 'Bandwagon', 'nan']


In [ ]:
y_train_actual

['Black-and-white Fallacy/Dictatorship',
 'Name calling/Labeling',
 'Smears',
 'Reductio ad hitlerum',
 'Transfer',
 'Appeal to fear/prejudice',
 'Loaded Language',
 'Slogans',
 'Causal Oversimplification',
 'Glittering generalities (Virtue)',
 'Flag-waving',
 "Misrepresentation of Someone's Position (Straw Man)",
 'Exaggeration/Minimisation',
 'Repetition',
 'Appeal to (Strong) Emotions',
 'Doubt',
 'Obfuscation',
 'Intentional vagueness',
 'Confusion',
 'Whataboutism',
 'Thought-terminating cliché',
 'Presenting Irrelevant Data (Red Herring)',
 'Appeal to authority',
 'Bandwagon',
 'nan']

In [ ]:
y_test_actual = list(train_dictionary.keys())
print(y_test_actual)

['Black-and-white Fallacy/Dictatorship', 'Name calling/Labeling', 'Smears', 'Reductio ad hitlerum', 'Transfer', 'Appeal to fear/prejudice', 'Loaded Language', 'Slogans', 'Causal Oversimplification', 'Glittering generalities (Virtue)', 'Flag-waving', "Misrepresentation of Someone's Position (Straw Man)", 'Exaggeration/Minimisation', 'Repetition', 'Appeal to (Strong) Emotions', 'Doubt', 'Obfuscation', 'Intentional vagueness', 'Confusion', 'Whataboutism', 'Thought-terminating cliché', 'Presenting Irrelevant Data (Red Herring)', 'Appeal to authority', 'Bandwagon', 'nan']


In [ ]:
for i, l in enumerate(train_df['labels']):
  l = str(l)
  if l == 'nan':
    train_df.loc[i, 'labels'] = 'nan'

In [ ]:
for i, l in enumerate(test_df['labels']):
  l = str(l)
  if l == 'nan':
    test_df.loc[i, 'labels'] = 'nan'

In [ ]:
npy_y_train = []
for l in train_df['labels']:
  l = str(l)
  arr = [0] * 24
  if l == 'nan':
    npy_y_train.append(arr)
    continue
  string = l.split(', ')
  for s in string:
    x = y_train_actual.index(s)
    arr[x] = 1
  npy_y_train.append(arr)

In [ ]:
npy_y_test = []
for l in test_df['labels']:
  l = str(l)
  arr = [0] * 24
  if l == 'nan':
    npy_y_test.append(arr)
    continue
  string = l.split(', ')
  for s in string:
    x = y_test_actual.index(s)
    arr[x] = 1
  npy_y_test.append(arr)

In [ ]:
len(npy_y_train)
len(npy_y_train)

687

In [ ]:
import numpy as np

np.save('train_labels.npy', npy_y_train)
np.save('test_labels.npy', npy_y_test)

In [ ]:
train_dictionary

{'Black-and-white Fallacy/Dictatorship': 19,
 'Name calling/Labeling': 252,
 'Smears': 450,
 'Reductio ad hitlerum': 15,
 'Transfer': 61,
 'Appeal to fear/prejudice': 66,
 'Loaded Language': 360,
 'Slogans': 45,
 'Causal Oversimplification': 31,
 'Glittering generalities (Virtue)': 84,
 'Flag-waving': 36,
 "Misrepresentation of Someone's Position (Straw Man)": 32,
 'Exaggeration/Minimisation': 60,
 'Repetition': 10,
 'Appeal to (Strong) Emotions': 68,
 'Doubt': 61,
 'Obfuscation': 1,
 'Intentional vagueness': 1,
 'Confusion': 1,
 'Whataboutism': 47,
 'Thought-terminating cliché': 20,
 'Presenting Irrelevant Data (Red Herring)': 2,
 'Appeal to authority': 19,
 'Bandwagon': 2,
 'nan': 29}

In [ ]:
test_dictionary


{'Name calling/Labeling': 65,
 'Slogans': 22,
 'Smears': 105,
 'Transfer': 23,
 'Appeal to (Strong) Emotions': 19,
 'Appeal to fear/prejudice': 18,
 'Loaded Language': 100,
 'Doubt': 41,
 'Exaggeration/Minimisation': 31,
 'Whataboutism': 14,
 'Reductio ad hitlerum': 7,
 'Obfuscation': 2,
 'Intentional vagueness': 2,
 'Confusion': 2,
 'Glittering generalities (Virtue)': 24,
 'Black-and-white Fallacy/Dictatorship': 7,
 'Flag-waving': 12,
 'Presenting Irrelevant Data (Red Herring)': 5,
 'Appeal to authority': 13,
 'Thought-terminating cliché': 6,
 'Repetition': 1,
 "Misrepresentation of Someone's Position (Straw Man)": 3,
 'Causal Oversimplification': 4,
 'Bandwagon': 1,
 'nan': 4}

In [ ]:
train_dictionary = {}
for l in train_df['labels']:
  l = str(l)
  string = l.split(', ')
  for s in string:
    train_dictionary[s] = train_dictionary.get(s, 0) + 1

In [ ]:
hello = []
whole_dir = '/content/drive/MyDrive/training_set_task3'
for im in train_df['image']:
  if str(im) == 'nan':
    print('nan')
  hello.append(f'{whole_dir}/{im}')

In [ ]:
bye = []
whole_dir = '/content/drive/MyDrive/test_set_task3'
for im in test_df['image']:
  if str(im) == 'nan':
    print('nan')
  bye.append(f'{whole_dir}/{im}')

In [ ]:
from PIL import Image
import numpy as np

X = []
for h in hello:
  img = Image.open(h)
  img = img.resize((256, 256))
  num_arr = np.array(img).tolist()
  X.append(num_arr)

Y = []
for  b in bye:
  img = Image.open(h)
  img = img.resize((256, 256))
  num_arr = np.array(img).tolist()
  Y.append(num_arr)

In [ ]:
np.save('train_img.npy', X)
np.save('test_img.npy', Y)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models import resnet18, ResNet18_Weights

from torch.optim import Adam, AdamW

import numpy as np
import os

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
X_train = np.load('train_img.npy')
X_test = np.load('test_img.npy')
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
X_train /= 255.0
X_test /= 255.0

y_train = np.load('train_labels.npy')
y_test = np.load('test_labels.npy')

In [ ]:
class Data(Dataset):
    def __init__(self, images, labels):
        self.length = len(labels)
        self.images = images
        self.labels = labels

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        image = torch.from_numpy(self.images[idx])
        label = torch.from_numpy(self.labels[idx])

        return image, label

In [ ]:
pin_memory = True if device == 'cuda' else False

train = Data(X_train, y_train)
test = Data(X_test, y_test)

train_dataloader = DataLoader(
    train,
    batch_size=4,
    shuffle=True,
    num_workers=1,
    pin_memory=pin_memory
)

test_dataloader = DataLoader(
    test,
    batch_size=4,
    shuffle=False,
    num_workers=1,
    pin_memory=pin_memory
)

In [ ]:
class TransferLearn(nn.Module):
    def __init__(self):
        super(TransferLearn, self).__init__()

        resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
        res = list(resnet.children())[:-1]
        self.res = nn.Sequential(*res)
        self.linear = nn.Linear(512, 24)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.permute(x, (0, 3, 1, 2))
        x = self.res(x)
        #print(x.shape)
        x = torch.squeeze(x)
        x = self.linear(x)
        #print(x.shape, "hello")
        x = self.sigmoid(x)

        return x

In [ ]:
model = TransferLearn().to(device)
#print(model)

In [ ]:
EPOCHS = 3

# Binary Cross Entropy since it's a binary array as output
loss_function = nn.BCELoss()
optimizer = Adam(model.parameters())

In [ ]:
for epoch in range(EPOCHS):
    train_loss_sum = 0
    model.train()

    # Training loop
    for i, (x_actual, y_actual) in enumerate(train_dataloader, 1):
        optimizer.zero_grad()
        x_actual, y_actual = x_actual.to(device), y_actual.type(torch.float32).to(device)
        y_pred = model(x_actual)
        loss = loss_function(y_pred, y_actual)
        train_loss_sum += loss.item()
        loss.backward()
        optimizer.step()

    val_loss_sum = 0
    model.eval()

    # Check validation loss to see if it's overfitting
    with torch.no_grad():
        for j, (x_actual, y_actual) in enumerate(test_dataloader, 1):
            x_actual, y_actual = x_actual.to(device), y_actual.type(torch.float32).to(device)
            y_pred = model(x_actual)
            loss = loss_function(y_pred, y_actual)
            val_loss_sum += loss.item()


    if epoch % 1 == 0:
        avg_train_loss = round(train_loss_sum / i, 3)
        avg_val_loss = round(val_loss_sum / j, 3)

        print(f'Training Loss: {avg_train_loss}')
        print(f'Validation Loss: {avg_val_loss}\n')
        folder_path = f'model/checkpoint-{epoch}'
        os.makedirs(folder_path, exist_ok=True)
        torch.save(model.state_dict(), f'{folder_path}/resnet18.pth')


In [ ]:
class TransferLearn(nn.Module):
    def __init__(self):
        super(TransferLearn, self).__init__()

        resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
        res = list(resnet.children())[:-1]
        self.res = nn.Sequential(*res)
        self.linear = nn.Linear(512, 24)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.permute(x, (0, 3, 1, 2))
        x = self.res(x)
        #print(x.shape)
        x = torch.squeeze(x)
        x = self.linear(x)
        #print(x.shape, "hello")
        x = self.sigmoid(x)

        return x

In [ ]:
# Load model for inference
checkpoint = 'model/checkpoint-2/resnet18.pth'
res_inference = TransferLearn()
res_inference.load_state_dict(torch.load(checkpoint))

In [ ]:
X_train = np.load('train_img.npy')
X_test = np.load('test_img.npy')
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
X_train /= 255.0
X_test /= 255.0

y_train = np.load('train_labels.npy')
y_test = np.load('test_labels.npy')

In [ ]:
class Data(Dataset):
    def __init__(self, images, labels):
        self.length = len(labels)
        self.images = images
        self.labels = labels

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        image = torch.from_numpy(self.images[idx])
        label = torch.from_numpy(self.labels[idx])

        return image, label

In [ ]:
pin_memory = True if device == 'cuda' else False

test = Data(X_test, y_test)

test_dataloader = DataLoader(
    test,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    pin_memory=pin_memory
)

In [ ]:
for i, (a, b) in enumerate(test_dataloader):
  print(a.shape, b.shape)
  break

In [ ]:
import torch
import torch.nn as nn

def find_metrics(test_dataloader):
    total_class = 0
    loss_sum = 0
    loss_function = nn.BCELoss()
    correct_count = 0
    class_correct = [0] * 24
    class_total = [0] * 24
    y_predict = []
    y_actually = []
    with torch.no_grad():
        for i, (x_actual, y_actual) in enumerate(test_dataloader, 1):
            x_actual, y_actual = x_actual.to(device), y_actual.type(torch.float32).to(device)
            y_pred = model(x_actual)
            y_pred = y_pred.view(-1, 24)
            # Apply threshold to convert probabilities to binary predictions for accuracy calculation
            y_pred_bin = (y_pred > 0.5).float()
            correct = (y_actual == y_pred_bin).sum().item()
            correct_count += correct
            total_class += y_actual.nelement()
            loss = loss_function(y_pred, y_actual)
            loss_sum += loss.item()

            y_predict.append(y_pred_bin.flatten().tolist())
            y_actually.append(y_actual.flatten().tolist())


    class_accuracy = [0] * 24
    for y_p, y_a in zip(y_predict, y_actually):
      for i in range(len(y_p)):
        #print(a, b)
        if y_p[i] == y_a[i]:
          class_accuracy[i] += 1


    for i in range(len(class_accuracy)):
      class_accuracy[i] /= len(test)

    #print(acc)

    avg_loss = loss_sum / len(test_dataloader)
    accuracy = correct_count / total_class
    return avg_loss, accuracy, class_accuracy

In [ ]:
avg_loss, accuracy, class_accuracy = find_metrics(test_dataloader)

print(f'Average Loss: {avg_loss}')
print(f'Accuracy: {accuracy}')
print(f'Class Accuracy: {class_accuracy}')